In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import train_test_split
from ml_metrics import rmsle
import xgboost as xgb
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model, preprocessing
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def evalerror(preds, dtrain):

    labels = dtrain.get_label()
    assert len(preds) == len(labels)
    labels = labels.tolist()
    preds = preds.tolist()
    terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0,preds[i]) + 1))
                    ** 2.0 for i,pred in enumerate(labels)]
    return 'error', (sum(terms_to_sum) * (1.0/len(preds))) ** 0.5

In [3]:
products = pd.read_csv("../input/producto_tabla.csv")
products.head()

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


In [4]:
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
# products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

products['short_name_processed'] = (products['short_name'].
                                    map(lambda x: " ".
                                        join([i for i in x.lower().split() 
                                              if i not in nltk.corpus.stopwords.words("spanish")])))
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (products['short_name_processed'].
                                    map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))

word_list = products['short_name_processed']

word_counter = {}
for word in word_list:
    if word in word_counter:
        word_counter[word] += 1
    else:
        word_counter[word] = 1

popular_words = sorted(word_counter, key = word_counter.get, reverse = True)

short_name_processed_list = popular_words[:10]


# short_name_processed_list = products['short_name_processed'].unique()

products = pd.concat([products.drop(['short_name', 'short_name_processed', 'NombreProducto'], axis=1),
                      pd.get_dummies(short_name_processed_list)], axis=1)
#products = products.drop([''], axis=1)

products.fillna(value=0, inplace=True)

print('products shape:', products.shape)

('products shape:', (2592, 13))


In [135]:
train = pd.read_csv('../input/train_1000000.csv', usecols=['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil'])
test = pd.read_csv('../input/test.csv', usecols=['id', 'Semana', 'Cliente_ID', 'Producto_ID'])
train.tail()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
999995,8,550065,43307,30
999996,8,2171180,43069,3
999997,8,2343367,43307,20
999998,8,39008,43316,2
999999,8,41116,43064,3


In [136]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
Semana               1000000 non-null int64
Cliente_ID           1000000 non-null int64
Producto_ID          1000000 non-null int64
Demanda_uni_equil    1000000 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


In [137]:
test.head()

,id,Semana,Cliente_ID,Producto_ID
0,0,11,4639078,35305
1,1,11,4705135,1238
2,2,10,4549769,32940
3,3,11,4717855,43066
4,4,11,966351,1277


In [138]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999251 entries, 0 to 6999250
Data columns (total 4 columns):
id             int64
Semana         int64
Cliente_ID     int64
Producto_ID    int64
dtypes: int64(4)
memory usage: 213.6 MB


In [139]:
grouping = ['Semana', 'Cliente_ID', 'Producto_ID']
train_group = train.groupby(grouping,as_index=False)['Demanda_uni_equil'].sum()

In [140]:
train_group.tail()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
998843,8,9788785,40447,16
998844,8,9808903,32393,14
998845,8,9887764,6469,6
998846,8,9892002,3270,5
998847,8,9991105,1700,47


In [141]:
train_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 998848 entries, 0 to 998847
Data columns (total 4 columns):
Semana               998848 non-null int64
Cliente_ID           998848 non-null int64
Producto_ID          998848 non-null int64
Demanda_uni_equil    998848 non-null int64
dtypes: int64(4)
memory usage: 38.1 MB


In [142]:
train_group.head()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,26,4767,42
1,3,107,43064,0
2,3,107,48996,56
3,3,906,4085,6
4,3,2010,1109,1


In [143]:
train_cliprod = train_group
train_cliprod["cliprod"] = train_cliprod["Cliente_ID"].map(str) + ' ' + train_cliprod["Producto_ID"].map(str)

In [144]:
test_cliprod = test
test_cliprod["cliprod"] = test_cliprod["Cliente_ID"].map(str) + ' ' + test_cliprod["Producto_ID"].map(str)

In [145]:
train_cliprod.head()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil,cliprod
0,3,26,4767,42,26 4767
1,3,107,43064,0,107 43064
2,3,107,48996,56,107 48996
3,3,906,4085,6,906 4085
4,3,2010,1109,1,2010 1109


In [146]:
test_cliprod.head()

,id,Semana,Cliente_ID,Producto_ID,cliprod
0,0,11,4639078,35305,4639078 35305
1,1,11,4705135,1238,4705135 1238
2,2,10,4549769,32940,4549769 32940
3,3,11,4717855,43066,4717855 43066
4,4,11,966351,1277,966351 1277


In [147]:
grouping = ['Semana', 'cliprod']
train_cliprod = train_cliprod.groupby(grouping,as_index=False)['Demanda_uni_equil'].sum()


In [148]:
train_cliprod.head()

,Semana,cliprod,Demanda_uni_equil
0,3,1000006 1230,3
1,3,1000006 4767,2
2,3,1000015 43285,4
3,3,1000025 43069,2
4,3,100003 36748,2


In [149]:
train_cliprod.tail()

,Semana,cliprod,Demanda_uni_equil
998843,8,99992 31471,5
998844,8,99992 43147,7
998845,8,999927 1182,3
998846,8,999927 32819,3
998847,8,99996 41938,2


In [150]:
s3 = train_cliprod[train_cliprod.Semana == 3].drop('Semana', axis=1).set_index('cliprod')
s4 = train_cliprod[train_cliprod.Semana == 4].drop('Semana', axis=1).set_index('cliprod')
s5 = train_cliprod[train_cliprod.Semana == 5].drop('Semana', axis=1).set_index('cliprod')
s6 = train_cliprod[train_cliprod.Semana == 6].drop('Semana', axis=1).set_index('cliprod')
s7 = train_cliprod[train_cliprod.Semana == 7].drop('Semana', axis=1).set_index('cliprod')
s8 = train_cliprod[train_cliprod.Semana == 8].drop('Semana', axis=1).set_index('cliprod')
s9 = train_cliprod[train_cliprod.Semana == 9].drop('Semana', axis=1).set_index('cliprod')

t10 = test_cliprod[test_cliprod.Semana == 10].drop('Semana', axis=1).set_index('cliprod')
t11 = test_cliprod[test_cliprod.Semana == 11].drop('Semana', axis=1).set_index('cliprod')

In [151]:
s3.head()

,Demanda_uni_equil
cliprod,
1000006 1230,3
1000006 4767,2
1000015 43285,4
1000025 43069,2
100003 36748,2


In [152]:
t10.head()

,id,Cliente_ID,Producto_ID
cliprod,,,
4549769 32940,2,4549769,32940
4414012 35305,7,4414012,35305
397854 1240,8,397854,1240
4387996 2233,11,4387996,2233
4446449 1240,13,4446449,1240


In [153]:
s4["d1"]=s3['Demanda_uni_equil']
s5["d1"]=s4['Demanda_uni_equil']
s6["d1"]=s5['Demanda_uni_equil']
s7["d1"]=s6['Demanda_uni_equil']
s8["d1"]=s7['Demanda_uni_equil']
s9["d1"]=s8['Demanda_uni_equil']

t10["d1"]=s9['Demanda_uni_equil']

s5["d2"]=s3['Demanda_uni_equil']
s6["d2"]=s4['Demanda_uni_equil']
s7["d2"]=s5['Demanda_uni_equil']
s8["d2"]=s6['Demanda_uni_equil']
s9["d2"]=s7['Demanda_uni_equil']

t10["d2"]=s8['Demanda_uni_equil']
t11["d2"]=s9['Demanda_uni_equil']

s6["d3"]=s3['Demanda_uni_equil']
s7["d3"]=s4['Demanda_uni_equil']
s8["d3"]=s5['Demanda_uni_equil']
s9["d3"]=s6['Demanda_uni_equil']

t10["d3"]=s7['Demanda_uni_equil']
t11["d3"]=s8['Demanda_uni_equil']

s7["d4"]=s3['Demanda_uni_equil']
s8["d4"]=s4['Demanda_uni_equil']
s9["d4"]=s5['Demanda_uni_equil']

t10["d4"]=s6['Demanda_uni_equil']
t11["d4"]=s7['Demanda_uni_equil']

s8["d5"]=s3['Demanda_uni_equil']
s9["d5"]=s4['Demanda_uni_equil']

t10["d5"]=s5['Demanda_uni_equil']
t11["d5"]=s6['Demanda_uni_equil']

s9["d6"]=s3['Demanda_uni_equil']

t10["d6"]=s4['Demanda_uni_equil']
t11["d6"]=s5['Demanda_uni_equil']

In [154]:
s3.fillna(value=0, inplace=True)
s4.fillna(value=0, inplace=True)
s5.fillna(value=0, inplace=True)
s6.fillna(value=0, inplace=True)
s7.fillna(value=0, inplace=True)
s8.fillna(value=0, inplace=True)
s9.fillna(value=0, inplace=True)

t10.fillna(value=0, inplace=True)
t11.fillna(value=0, inplace=True)




In [155]:
s3["d1"]=np.nan

t11["d1"]=np.nan


s3["d2"]=np.nan
s4["d2"]=np.nan



s3["d3"]=np.nan
s4["d3"]=np.nan
s5["d3"]=np.nan



s3["d4"]=np.nan
s4["d4"]=np.nan
s5["d4"]=np.nan
s6["d4"]=np.nan


s3["d5"]=np.nan
s4["d5"]=np.nan
s5["d5"]=np.nan
s6["d5"]=np.nan
s7["d5"]=np.nan



s3["d6"]=np.nan
s4["d6"]=np.nan
s5["d6"]=np.nan
s6["d6"]=np.nan
s7["d6"]=np.nan
s8["d6"]=np.nan



In [156]:
s3.head()

,Demanda_uni_equil,d1,d2,d3,d4,d5,d6
cliprod,,,,,,,
1000006 1230,3,NaN,NaN,NaN,NaN,NaN,NaN
1000006 4767,2,NaN,NaN,NaN,NaN,NaN,NaN
1000015 43285,4,NaN,NaN,NaN,NaN,NaN,NaN
1000025 43069,2,NaN,NaN,NaN,NaN,NaN,NaN
100003 36748,2,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
t10.head()

,id,Cliente_ID,Producto_ID,d1,d2,d3,d4,d5,d6
cliprod,,,,,,,,,
4549769 32940,2,4549769,32940,0.0,0.0,0.0,0.0,0.0,0.0
4414012 35305,7,4414012,35305,0.0,0.0,0.0,0.0,0.0,0.0
397854 1240,8,397854,1240,0.0,0.0,0.0,0.0,0.0,0.0
4387996 2233,11,4387996,2233,0.0,0.0,0.0,0.0,0.0,0.0
4446449 1240,13,4446449,1240,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
s3 = s3.reset_index()
s3.head()

,cliprod,Demanda_uni_equil,d1,d2,d3,d4,d5,d6
0,1000006 1230,3,NaN,NaN,NaN,NaN,NaN,NaN
1,1000006 4767,2,NaN,NaN,NaN,NaN,NaN,NaN
2,1000015 43285,4,NaN,NaN,NaN,NaN,NaN,NaN
3,1000025 43069,2,NaN,NaN,NaN,NaN,NaN,NaN
4,100003 36748,2,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
s3['Cliente_ID'], s3['Producto_ID'] = zip(*s3['cliprod'].apply(lambda x: x.split(' ', 1)))

In [160]:
s3.head()

,cliprod,Demanda_uni_equil,d1,d2,d3,d4,d5,d6,Cliente_ID,Producto_ID
0,1000006 1230,3,NaN,NaN,NaN,NaN,NaN,NaN,1000006,1230
1,1000006 4767,2,NaN,NaN,NaN,NaN,NaN,NaN,1000006,4767
2,1000015 43285,4,NaN,NaN,NaN,NaN,NaN,NaN,1000015,43285
3,1000025 43069,2,NaN,NaN,NaN,NaN,NaN,NaN,1000025,43069
4,100003 36748,2,NaN,NaN,NaN,NaN,NaN,NaN,100003,36748


In [161]:
s3 = s3.drop(['cliprod'], axis=1)
s3.head()

,Demanda_uni_equil,d1,d2,d3,d4,d5,d6,Cliente_ID,Producto_ID
0,3,NaN,NaN,NaN,NaN,NaN,NaN,1000006,1230
1,2,NaN,NaN,NaN,NaN,NaN,NaN,1000006,4767
2,4,NaN,NaN,NaN,NaN,NaN,NaN,1000015,43285
3,2,NaN,NaN,NaN,NaN,NaN,NaN,1000025,43069
4,2,NaN,NaN,NaN,NaN,NaN,NaN,100003,36748


In [162]:
s4 = s4.reset_index()
s4['Cliente_ID'], s4['Producto_ID'] = zip(*s4['cliprod'].apply(lambda x: x.split(' ', 1)))
s4 = s4.drop(['cliprod'], axis=1)

s5 = s5.reset_index()
s5['Cliente_ID'], s5['Producto_ID'] = zip(*s5['cliprod'].apply(lambda x: x.split(' ', 1)))
s5 = s5.drop(['cliprod'], axis=1)

s6 = s6.reset_index()
s6['Cliente_ID'], s6['Producto_ID'] = zip(*s6['cliprod'].apply(lambda x: x.split(' ', 1)))
s6 = s6.drop(['cliprod'], axis=1)

s7 = s7.reset_index()
s7['Cliente_ID'], s7['Producto_ID'] = zip(*s7['cliprod'].apply(lambda x: x.split(' ', 1)))
s7 = s7.drop(['cliprod'], axis=1)

s8 = s8.reset_index()
s8['Cliente_ID'], s8['Producto_ID'] = zip(*s8['cliprod'].apply(lambda x: x.split(' ', 1)))
s8 = s8.drop(['cliprod'], axis=1)

s9 = s9.reset_index()
s9['Cliente_ID'], s9['Producto_ID'] = zip(*s9['cliprod'].apply(lambda x: x.split(' ', 1)))
s9 = s9.drop(['cliprod'], axis=1)


t10 = t10.reset_index()
t10['Cliente_ID'], t10['Producto_ID'] = zip(*t10['cliprod'].apply(lambda x: x.split(' ', 1)))
t10 = t10.drop(['cliprod'], axis=1)

t11 = t11.reset_index()
t11['Cliente_ID'], t11['Producto_ID'] = zip(*t11['cliprod'].apply(lambda x: x.split(' ', 1)))
t11 = t11.drop(['cliprod'], axis=1)

In [163]:
t10.head()

,id,Cliente_ID,Producto_ID,d1,d2,d3,d4,d5,d6
0,2,4549769,32940,0.0,0.0,0.0,0.0,0.0,0.0
1,7,4414012,35305,0.0,0.0,0.0,0.0,0.0,0.0
2,8,397854,1240,0.0,0.0,0.0,0.0,0.0,0.0
3,11,4387996,2233,0.0,0.0,0.0,0.0,0.0,0.0
4,13,4446449,1240,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
train_joined = pd.concat([s3, s4, s5, s6, s7, s8, s9], ignore_index = True)
test_joined = pd.concat([t10, t11], ignore_index = True)

In [165]:
train_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1073267 entries, 0 to 1073266
Data columns (total 9 columns):
Demanda_uni_equil    1073267 non-null float64
d1                   880443 non-null float64
d2                   689248 non-null float64
d3                   505016 non-null float64
d4                   328856 non-null float64
d5                   148838 non-null float64
d6                   74419 non-null float64
Cliente_ID           1073267 non-null object
Producto_ID          1073267 non-null object
dtypes: float64(7), object(2)
memory usage: 81.9+ MB


In [166]:
train_joined.head()

,Demanda_uni_equil,d1,d2,d3,d4,d5,d6,Cliente_ID,Producto_ID
0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1000006,1230
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1000006,4767
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1000015,43285
3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1000025,43069
4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,100003,36748


In [167]:
train_joined[['Cliente_ID', 'Producto_ID']]=train_joined[['Cliente_ID', 'Producto_ID']].astype('int32')
test_joined[['Cliente_ID', 'Producto_ID']]=test_joined[['Cliente_ID', 'Producto_ID']].astype('int32')

In [168]:
train_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1073267 entries, 0 to 1073266
Data columns (total 9 columns):
Demanda_uni_equil    1073267 non-null float64
d1                   880443 non-null float64
d2                   689248 non-null float64
d3                   505016 non-null float64
d4                   328856 non-null float64
d5                   148838 non-null float64
d6                   74419 non-null float64
Cliente_ID           1073267 non-null int32
Producto_ID          1073267 non-null int32
dtypes: float64(7), int32(2)
memory usage: 73.7 MB


In [169]:
test_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999251 entries, 0 to 6999250
Data columns (total 9 columns):
Cliente_ID     int32
Producto_ID    int32
d1             float64
d2             float64
d3             float64
d4             float64
d5             float64
d6             float64
id             int64
dtypes: float64(6), int32(2), int64(1)
memory usage: 480.6 MB


In [170]:
train_joined.to_csv('../input/train_alld.csv')
test_joined.to_csv('../input/test_alld.csv')
train_joined.fillna(value=-999, inplace=True)
test_joined.fillna(value=-999, inplace=True)

In [171]:
# Join data and products
train_joined = train_joined.join(products, on='Producto_ID', lsuffix='_t')
train_joined.fillna(value=0, inplace=True)

print('train data joined')

test_joined = test_joined.join(products, on='Producto_ID', lsuffix='_t')
test_joined.fillna(value=0, inplace=True)

print('test data joined')
print('train shape', train_joined.shape)
print('test shape', test_joined.shape)

train data joined
test data joined
('train shape', (1073267, 22))
('test shape', (6999251, 22))


In [172]:
ids = test_joined['id']
test_joined = test_joined.drop(['id'],axis = 1)

y = train_joined['Demanda_uni_equil']
X = train_joined[test_joined.columns.values]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1729)

print ('Division_Set_Shapes:', X.shape, y.shape)
print ('Validation_Set_Shapes:', X_train.shape, X_test.shape)

('Division_Set_Shapes:', (1073267, 21), (1073267,))
('Validation_Set_Shapes:', (858613, 21), (214654, 21))


In [173]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858613 entries, 753494 to 1050253
Data columns (total 21 columns):
Cliente_ID        858613 non-null int32
Producto_ID_t     858613 non-null int32
d1                858613 non-null float64
d2                858613 non-null float64
d3                858613 non-null float64
d4                858613 non-null float64
d5                858613 non-null float64
d6                858613 non-null float64
Producto_ID       858613 non-null float64
weight            858613 non-null float64
pieces            858613 non-null float64
canelit           858613 non-null float64
delici chochit    858613 non-null float64
delici vainill    858613 non-null float64
gansit            858613 non-null float64
medi noch         858613 non-null float64
pan blanc         858613 non-null float64
pan integral      858613 non-null float64
pinguin           858613 non-null float64
tortillin         858613 non-null float64
wond              858613 non-null float64
dtype

In [174]:
params = {}
params['objective'] = "reg:linear"
params['eta'] = 0.025
params['max_depth'] = 5
params['subsample'] = 0.8
params['colsample_bytree'] = 0.6
params['silent'] = True

In [175]:
test_preds = np.zeros(test_joined.shape[0])
xg_train = xgb.DMatrix(X_train, label=y_train, missing = -999)
xg_test = xgb.DMatrix(X_test)

watchlist = [(xg_train, 'train_joined')]
num_rounds = 100

xgclassifier = xgb.train(params, xg_train, num_rounds, watchlist, feval = evalerror,
                         early_stopping_rounds= 20, verbose_eval = 10)
preds = xgclassifier.predict(xg_test, ntree_limit=xgclassifier.best_iteration)

print ('RMSLE Score:', rmsle(y_test, preds))

Will train until train_joined error hasn't decreased in 20 rounds.
[0]	train_joined-error:1.328513
[10]	train_joined-error:0.922342
[20]	train_joined-error:0.812183
[30]	train_joined-error:0.792219
[40]	train_joined-error:0.806088


('RMSLE Score:', nan)


[50]	train_joined-error:0.823264
Stopping. Best iteration:
[30]	train_joined-error:0.792219



In [176]:
fxg_test = xgb.DMatrix(test_joined)
fold_preds = np.around(xgclassifier.predict(fxg_test, ntree_limit=xgclassifier.best_iteration), decimals = 1)
test_preds += fold_preds

In [177]:
submission = pd.DataFrame({'id':ids, 'Demanda_uni_equil': test_preds})

In [178]:
submission.Demanda_uni_equil = submission.Demanda_uni_equil.round()

In [179]:
submission[["id","Demanda_uni_equil"]].to_csv('../submissions/' +
                                              datetime.now().strftime('%Y-%m-%d-%H-%M-%S') +'.csv', index=False)

print ('done')

done
